# Floating points errors

In [2]:
import math
import numpy

> **Problem**
>
>Why does the following Python code
>
>```python
>0.1 + 0.2 + 0.3 == 0.6
>```
>
>return `False`? Let's try it out for ourself

In [4]:
0.1 + 0.2 + 0.3 == 0.6

False

Yep, that does appear strange.

---

## Represening real numbers ($\mathbb R$) in a computer. 

We learnt that we should represent a real numbers in Python using the type `float`. 

Computers (and calculators) have a finite amount of memory. Modern computers may have upwards of 4 GB ($4 \times 10^9$ bytes) of RAM. This memory is used to run your computers operating systems and other applications.

Since the total memory available on a computer is finite, so to is the amount of memory (in bytes) availble to represent a real number. Your Python code (as an example of an application) will store many real numbers, for example

```python
N = 100
x = numpy.zeros(N)
```

requires storing 100 real numbers. For this reason, the amount of memory assigned to represent a real number as `float` is kept as small as possible. 

The consequence of using a finite amount of memory to represent a real number via a `float` is twofold:

1. The numbers possible to represent in a computer with `float` cannot be arbitrarly large of arbitrarily small.
2. There must be gaps between the represented numbers.

Points 1. and 2. are fundemental limitations of representing real numbers in a computer.

### Ranges

The so called "double precision" standard (defined by the IEEE-754 arithmetic standard) allows: 
* numbers as large as $n_\text{max} = 1.79 \times 10^{308}$ to be represented;
* numbers as small as $n_\text{min} = 2.23 \times 10^{-308}$ to be represented.

If you try to assign a value to a double precision number less than $n_\text{min}$ *underflow* occurs

In [14]:
x = 2.23e-308 # This is fine
print(x)

2.23e-308


In [16]:
x = 1.23e-308 # Also appears to be fine
print(x)

1.23e-308


Hardward engineers use a sub-normal representation for small numbers allow numbers (for double precision to) to actually be as small as $2.5 \times 10^{-324}$.

In [19]:
x = 2.40000000e-324 # This is truncated to zero
print(x, type(x))

0.0 <class 'float'>


Here underflow results in the number being truncated to zero.

If you try to assign a value to a double precision number greater than $n_\text{max}$ *overflow* occurs.

In [23]:
x = 1.79e308 # This is fine
print(x)

1.79e+308


However exceeding $n_\text{max}$ results in `x` being assigned a value of `inf`, which means "infinity". It is not the true infinity, it is the infinity associated with double precision representation.

In [26]:
x = 1.799e308
print(x)

inf


### How numbers are stored

Python's `float` class uses use a double precision representation.
We saw earlier that this is equivalent to NumPy's `float64`.
NumPy supports lower precision numbers such as `float32` and `float16`.

Numbers in a computer are stored in scientific notation, e.g.
$$
    +1.12132343678e-64
$$

The memory (bytes) used to represent a floating point number are partitioned into representing three parts of the number:
* Storage for the sign of the number (`+`);
* Storage for the exponent (`-64`);
* Storage for the decimal digits `1.12132343678` (or mantissa).

The `16`, `32` and `64` in `float16, float32, float64` indicate the number of bits used to store the number. There are 8 bits in a byte.

For `float16` we have
* 1 bit is used for the sign of the number
* 5 bits are are used for the exponent;
* 10 bits are used for the decimal digits.
    - This translates to a precision of 7 decimal digits.

For `float32` we have
* 1 bit is used for the sign of the number
* 8 bits are are used for the exponent;
* 23 bits are used for the decimal digits.
    - This translates to a precision of approximately 7 decimal digits.

For `float64` we have
* 1 bit is used for the sign of the number
* 11 bits are are used for the exponent;
* 32 bits are used for the decimal digits.
    - This translates to a precision of approximately 16 decimal digits.

Historically computers were shipped with `float32`. Later `float64` become availble. Since a `float64` uses twice as many bits in its representation compared to the original floating point unit (`float32`) they become known as "double precision". Recently in ML there is interest in `float16` - following the same naming convention these become known as "half-precision". These days `float128` is used and these are referred to as "quad-precision" floating point representations.

Python's `float` class uses use a double precision representation.
We saw earlier that this is equivalent to NumPy's `float64`.
NumPy supports lower precision numbers such as `float32` and `float16`.
The consequence of using a lower precision number representation is the range of possible numbers you can represent is smaller and gaps between representable numbers is larger.

### Gappiness

The interval of real numbers $[1, 2]$ when represented via double precision is given by the discrete subset

$$
1, \, 1 + 2^{-52}, \, 1 + 2 \times 2^{-52}, \, 1 + 3 \times 2^{-52}, \, \dots, \, 2
$$

The interval $[2, 4]$ is given by the discrete subset

$$
2, \, 2 + 2^{-52}, \, 2 + 2 \times 2^{-52}, \, 2 + 3 \times 2^{-52}, \, \dots, \, 4
$$

In a relative sense, the gaps between adjacent numbers is $2^{-52} \approx 2.22 \times 10^{-16}$.

---

We now what a sense of what the answer to the problem stated at the top of the document is.

Think of the problem this way

```python
float(0.1) + float(0.2) + float(0.3) 
```

We know the `float` representation of numbers contains gaps. This means not every single real number in $\mathbb R$ can be represented! Hence if `float(0.1)` can only be approximately represented, say within $2.22 \times 10^{-16}$ and the same is true for `float(0.2)` and `float(0.3)` then when we add them all up we will not end up with a result which is exactly 0.6. We likely have $0.6 + O(10^{-16})$.

Lets confirm this

In [35]:
result = float(0.1) + float(0.2) + float(0.3)

print('result', ('%1.20e'%result))

# Let's compute the difference between what we expected (0.6) and result

delta = result - 0.6

print('delta', ('%1.20e'%delta))

result 6.00000000000000088818e-01
delta 1.11022302462515654042e-16


Inded the result is not **exactly** 0.6 and the difference is $O(10^{-16})$.

---

## Round-off Error

If numbers are not exactly represented and you perform operations with them, the resulting quantity will also have errors. 

Take `z = a + b`. If there is a representation error associated with number stored in `a`, and in `b`, `a + b` will have an error. There is the additional which may occur in that the resulting value for `a + b` can also not be exactly represent so another representation error creeps in.

The more operations you perform, the worse the error in the final result will become. This is called round-off error.

Consider the following piece of code (modified from "PyNumMeth", Chapter 9)

In [41]:
def add_and_subtract(iterations):
    result = 1.0
    
    for i in range(iterations):
        result += 0.333

    for i in range(iterations):
        result -= 0.333
    return result

In exact arithmetic (infinite precision) the returned value `result` should always equal exactly `1.0`, independent of the value of `iterations`. We now understand that this probably won't be the case with double precision numbers, even if `iterations = 1`.

In [44]:
r = add_and_subtract(30)
print('r', ('%1.40e')%r)

r 9.9999999999999911182158029987476766109467e-01


In [46]:
r = add_and_subtract(100)
print('r', ('%1.40e')%r)

r 9.9999999999999733546474089962430298328400e-01


In [48]:
r = add_and_subtract(1000)
print('r', ('%1.40e')%r)

r 1.0000000000000328626015289046335965394974e+00


In [50]:
r = add_and_subtract(100000)
print('r', ('%1.40e')%r)

r 1.0000000000027613467068476893473416566849e+00


In [52]:
for k in [30, 100, 1000, 100000, 100000000]:
    r = add_and_subtract(k)
    err = math.fabs(r - 1.0)
    print('k =', (('%10d')%k), 'error =', (('%1.4e')%err))


k =         30 error = 8.8818e-16
k =        100 error = 2.6645e-15
k =       1000 error = 3.2863e-14
k =     100000 error = 2.7613e-12
k =  100000000 error = 2.2746e-09


As expected, the more times you perform the add and subtract operations, the more errors you accumulate.

The growth of round-off is related to the precision (or number of decimal digits). With fewer decimal digits, round-off errors will grow faster.

We repeat our round-off error experiment below with a `float16` variant.

In [56]:
# float16 version of `add_and_subtract`.
def add_and_subtract_f16(iterations):
    result = numpy.float16(1.0) # force float16 to be used
    
    for i in range(iterations):
        result += numpy.float16(0.333) # force float16 to be used

    for i in range(iterations):
        result -= numpy.float16(0.333) # force float16 to be used
    return result

In [57]:
for k in [30, 100, 1000, 100000, 100000000]:
    r_float16 = add_and_subtract_f16(k)
    err = math.fabs(r_float16 - 1.0)
    print('k =', (('%10d')%k), 'error =', (('%1.4e')%err))


k =         30 error = 4.8828e-03
k =        100 error = 1.2695e-02
k =       1000 error = 1.6797e-01
k =     100000 error = 1.0250e+03
k =  100000000 error = 1.0250e+03


It doesn't take long for the calculation to be completely wrong due to round-off errors.

### Catestrophic cancellation

In numerical analysis, catastrophic cancellation (somtimes called subtractive cancellation error) is the phenomenon that subtracting good approximations to two very nearby numbers may yield a very bad approximation to the difference of the original numbers.

Subtracting two floating-point numbers that are very close together leaves very few significant digits — a great deal of information is lost. Lower precision numbers have less available digits so the problem is made worse.

Here is a simple example showing the problem
Consider computing $x - y$ using floats (7 decimal digits of accuracy)

In [65]:
x = numpy.float32(1.123004)
y = numpy.float32(1.123025)

v1 = x - y
print('x - y =', '%1.6f'%v1)

x - y = -0.000021


Our original number had 7 decimal digits, but the difference on has 2.
This illustrates that subtracting two floating-point numbers that are very close together leaves very few significant digits — a great deal of information is lost. As the true value is very small, the round-off error becomes much more significant, and sometimes will become much larger than the value being computed

Another common example is solving a quadratic equation, i.e.

$$
a x^2 + b x + c = 0
$$

we often quote the way to obtain solutions is to use the formula

$$
x = \frac{-b \pm \sqrt{b^2 - 4ac} }{2a}.
$$

Yielding
$$
x_1 = \frac{-b + \sqrt{b^2 - 4ac} }{2a}.
$$
$$
x_2 = \frac{-b - \sqrt{b^2 - 4ac} }{2a}.
$$

Let's suppose that $b > 0$. In this case,
there are not subtractive cancellation errors associated with $x_2$ as two terms on the numerator have the sign.

However, $x_1$ may be subject to subtractive cancellation error if $4ac$ is very small. There is a simple solution to avoid cancellation

$$
x_1 = \frac{-b + \sqrt{b^2 - 4ac} }{2a} \times \frac{-b - \sqrt{b^2 - 4ac} }{-b - \sqrt{b^2 - 4ac}}
$$

$$
x_1 = \frac{b^2 - (b^2 - 4ac) }{2a(-b - \sqrt{b^2 - 4ac})}
= \frac{2c}{(-b - \sqrt{b^2 - 4ac})}
= \frac{c}{a x_2}
$$

The above method to compute $x_1$ does not involve subtracting two numbers so there is no risk of catestrophic cancellation.